In [1]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/Users/lipang/Library/Python/3.8/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 1.51MB/s]
Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 7.80kB/s]
Downloading: 100%|██████████| 570/570 [00:00<00:00, 104kB/s]


In [2]:
sentences_list = []

file1 = open('house_mouse.txt', 'r')
sentences1 = file1.readlines()
  
for sentence in sentences1:
    sentences_list.append(sentence.strip())

file1 = open('computer_mouse.txt', 'r')
sentences2 = file1.readlines()

for sentence in sentences2:
    sentences_list.append(sentence.strip())

In [6]:
tokenized_sentences = []

for sentence in sentences_list:
    marked_text = "[CLS] " + sentence + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    tokenized_sentences.append(tokenized_text)

print(tokenized_sentences)

[['[CLS]', 'the', 'house', 'mouse', 'is', 'a', 'small', 'mammal', 'of', 'the', 'order', 'rode', '##nti', '##a', ',', 'characteristic', '##ally', 'having', 'a', 'pointed', 'snout', ',', 'large', 'rounded', 'ears', ',', 'and', 'a', 'long', 'and', 'almost', 'hair', '##less', 'tail', '.', '[SEP]'], ['[CLS]', 'although', 'a', 'wild', 'animal', ',', 'the', 'house', 'mouse', 'has', 'benefited', 'significantly', 'from', 'ass', '##oc', '##iating', 'with', 'human', 'habitat', '##ion', 'to', 'the', 'point', 'that', 'truly', 'wild', 'populations', 'are', 'significantly', 'less', 'common', 'than', 'the', 'semi', '-', 'tame', 'populations', 'near', 'human', 'activity', '.', '[SEP]'], ['[CLS]', 'the', 'house', 'mouse', 'has', 'been', 'domestic', '##ated', 'as', 'the', 'pet', '.', '[SEP]'], ['[CLS]', 'the', 'average', 'sleep', 'time', 'of', 'a', 'captive', 'house', 'mouse', 'is', 'reported', 'to', 'be', '12', '.', '5', 'hours', 'per', 'day', '.', '[SEP]'], ['[CLS]', 'some', 'populations', 'are', 'hybr

In [48]:
embeddings_list = []

model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
model.eval()

for tokenized_sentence in tokenized_sentences:

    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_sentence)
    segments_ids = [1] * len(tokenized_sentence)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

        '''
        hidden_states has 4 dimensions:
        1. The layer number (13 layers)
        2. The batch number (sentences)
        3. The word / token number (tokens in sentence)
        4. The hidden unit / feature number (768 features) 
        '''

    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)

    embeddings_list.append(token_embeddings)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [49]:
embeddings_list = np.array(embeddings_list)
embeddings_list[0][12].shape

/var/folders/v4/3zx43p8s5t311plcz6xsj0yw0000gn/T/ipykernel_17803/394597466.py:1: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  embeddings_list = np.array(embeddings_list)
/var/folders/v4/3zx43p8s5t311plcz6xsj0yw0000gn/T/ipykernel_17803/394597466.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  embeddings_list = np.array(embeddings_list)


torch.Size([36, 768])